In [224]:
import numpy as np
import pandas as pd
from xgboost import XGBRegressor


from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import AdaBoostRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingRegressor
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn import metrics


In [169]:
data = pd.read_csv("CE9010_cleaned_data.csv")
data.tail(5)

,title,price,brand,resolution,screen_size,level_1_model,processor,processor_speed,ram,warranty,link,description,weight,hdd_gb,ssd_gb,graphic_card,hdd_exist,ssd_exist,graphic_card_exist,storage
297,refurbished lenovo thinkpad x280 8th 12.5'' / ...,868.0,lenovo,fhd,12.5,thinkpad,intel core i5-8350u,1.7,8.0,0,https://www.reebelo.com/collections/all/produc...,NaN,1.27,NaN,256.0,intel uhd 620,0,1,1,256.0
298,refurbished hp elitebook 840 g3 / 14'' / core ...,469.0,hp,hd,12.5,elitebook,intel core i5-6300u,2.4,8.0,0,https://www.reebelo.com/collections/all/produc...,NaN,1.54,NaN,256.0,intel hd 520,0,1,1,256.0
299,refurbished lenovo thinkpad x1 yoga 1st gen ul...,849.0,lenovo,fhd,14.0,thinkpad,intel core i7-6600u,2.6,16.0,0,https://www.reebelo.com/collections/all/produc...,NaN,1.36,NaN,256.0,intel hd 520,0,1,1,256.0
300,refurbished lenovo thinkpad x1 carbon 6th gen ...,1068.0,lenovo,fhd,14.0,thinkpad,intel core i5-8350u,1.7,8.0,0,https://www.reebelo.com/collections/all/produc...,NaN,1.13,NaN,256.0,intel uhd 620,0,1,1,256.0
301,refurbished hp elitebook 820 g3 / 12.5'' / cor...,399.0,hp,hd,12.5,elitebook,intel core i5-6300u,2.4,8.0,0,https://www.reebelo.com/collections/all/produc...,NaN,1.54,NaN,128.0,intel hd 520,0,1,1,128.0


In [20]:
data['resolution'].unique()


array(['wqxga', 'fhd', 'hd+', 'sxga', 'hd', 'xga', 'qhd', 'vga', 'qwxga+',
       'fhd+', 'qhd+', 'wxga+', 'wqhd', 'uhd'], dtype=object)

In [170]:
data_encoded = data.copy()

#one hot encoding for brands
dummies = pd.get_dummies(data_encoded['brand']).rename(columns=lambda x: 'brand_' + str(x))
data_encoded = pd.concat([data_encoded, dummies], axis=1)

#one hot encoding for level_1_model
dummies = pd.get_dummies(data_encoded['level_1_model']).rename(columns=lambda x: 'level_1_model_' + str(x))
data_encoded = pd.concat([data_encoded, dummies], axis=1)
data_encoded.tail(5)

#encoding resolution
data_encoded['resolution'][data.resolution == 'vga'] = 1
data_encoded['resolution'][data.resolution == 'xga'] = 2
data_encoded['resolution'][data.resolution == 'hd'] = 3
data_encoded['resolution'][data.resolution == 'wxga+'] = 4
data_encoded['resolution'][data.resolution == 'sxga'] = 5
data_encoded['resolution'][data.resolution == 'hd+'] = 6
data_encoded['resolution'][data.resolution == 'fhd'] = 7
data_encoded['resolution'][data.resolution == 'fhd+'] = 8
data_encoded['resolution'][data.resolution == 'qhd'] = 9
data_encoded['resolution'][data.resolution == 'wqhd'] = 10
data_encoded['resolution'][data.resolution == 'wqxga'] = 11
data_encoded['resolution'][data.resolution == 'qwxga+'] = 12
data_encoded['resolution'][data.resolution == 'qhd+'] = 13
data_encoded['resolution'][data.resolution == 'uhd'] = 14

#remove unnecessary variables
data_encoded = data_encoded.drop('brand',1)
data_encoded = data_encoded.drop('level_1_model',1)
data_encoded = data_encoded.drop('title',1)
data_encoded = data_encoded.drop('processor',1)
data_encoded = data_encoded.drop('link',1)
data_encoded = data_encoded.drop(['description','graphic_card','weight','hdd_gb', 'ssd_gb'],1)


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
C:\ProgramData\Anaconda3\lib\site-pack

In [171]:
data_encoded.columns

Index(['price', 'resolution', 'screen_size', 'processor_speed', 'ram',
       'warranty', 'hdd_exist', 'ssd_exist', 'graphic_card_exist', 'storage',
       'brand_acer', 'brand_aftershock', 'brand_apple', 'brand_asus',
       'brand_asuspro', 'brand_compaq', 'brand_dell', 'brand_fujitsu',
       'brand_gigabyte', 'brand_hp', 'brand_lenovo', 'brand_medion',
       'brand_microsoft', 'brand_msi', 'brand_panasonic', 'brand_razer',
       'brand_siemens', 'brand_toshiba', 'level_1_model_aero',
       'level_1_model_apex', 'level_1_model_armada', 'level_1_model_aspire',
       'level_1_model_blade', 'level_1_model_bravo',
       'level_1_model_chromebook', 'level_1_model_elite',
       'level_1_model_elitebook', 'level_1_model_envy', 'level_1_model_erazer',
       'level_1_model_fx502vm-dm266t', 'level_1_model_ge72',
       'level_1_model_gf75', 'level_1_model_ideapad', 'level_1_model_inpiron',
       'level_1_model_inspiron', 'level_1_model_laptop',
       'level_1_model_latitude', 'level_

In [206]:
data_encoded['resolution'] = data_encoded['resolution'].apply(pd.to_numeric)


KeyError: 'resolurion'

In [207]:
data_encoded['resolution']

0      11
1       7
2       6
3       5
4       5
       ..
297     7
298     3
299     7
300     7
301     3
Name: resolution, Length: 302, dtype: int64

In [172]:
#Normalization
data_encoded.describe()

,price,screen_size,processor_speed,ram,warranty,hdd_exist,ssd_exist,graphic_card_exist,storage,brand_acer,...,level_1_model_travelmate,level_1_model_travelmate p248-m,level_1_model_vivobook,level_1_model_vostro,level_1_model_x201,level_1_model_xg15-v2,level_1_model_xps,level_1_model_yoga,level_1_model_zbook,level_1_model_zenbook
count,302.000000,302.000000,302.000000,302.000000,302.000000,302.000000,302.000000,302.000000,302.000000,302.000000,...,302.000000,302.000000,302.000000,302.000000,302.000000,302.000000,302.000000,302.000000,302.000000,302.000000
mean,931.136556,13.892053,2.234669,9.065915,0.798013,0.211921,0.761589,0.582781,383.660927,0.026490,...,0.003311,0.003311,0.026490,0.009934,0.003311,0.003311,0.003311,0.013245,0.023179,0.039735
std,641.565472,1.161625,0.602418,6.287541,0.402149,0.409347,0.426819,0.493918,334.802832,0.160854,...,0.057544,0.057544,0.160854,0.099337,0.057544,0.057544,0.057544,0.114512,0.150721,0.195660
min,129.000000,10.600000,0.150000,0.031250,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,449.250000,13.300000,1.800000,4.000000,1.000000,0.000000,1.000000,0.000000,256.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,792.380000,14.000000,2.300000,8.000000,1.000000,0.000000,1.000000,1.000000,256.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,1225.635000,14.000000,2.600000,8.000000,1.000000,0.000000,1.000000,1.000000,512.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,3999.000000,17.300000,4.700000,64.000000,1.000000,1.000000,1.000000,1.000000,2048.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [56]:
#NORMALIZATION

#Z SCORING
data_z=data_encoded.copy()
data_z[['screen_size', 'processor_speed','ram','storage']] = (data_z[['screen_size', 'processor_speed','ram','storage']] - data_z[['screen_size', 'processor_speed','ram','storage']].mean())/data_z[['screen_size', 'processor_speed','ram','storage']].std()
data_z.describe()

#Max-min
data_n=data_encoded.copy()
data_n[['screen_size', 'processor_speed','ram','storage']] = (data_n[['screen_size', 'processor_speed','ram','storage']] - data_n[['screen_size', 'processor_speed','ram','storage']].min())/(data_n[['screen_size', 'processor_speed','ram','storage']].max() - data_n[['screen_size', 'processor_speed','ram','storage']].min())
data_n.describe()

,price,screen_size,processor_speed,ram,warranty,hdd_exist,ssd_exist,graphic_card_exist,storage,brand_acer,...,level_1_model_travelmate,level_1_model_travelmate p248-m,level_1_model_vivobook,level_1_model_vostro,level_1_model_x201,level_1_model_xg15-v2,level_1_model_xps,level_1_model_yoga,level_1_model_zbook,level_1_model_zenbook
count,302.000000,302.000000,302.000000,302.000000,302.000000,302.000000,302.000000,302.000000,302.000000,302.000000,...,302.000000,302.000000,302.000000,302.000000,302.000000,302.000000,302.000000,302.000000,302.000000,302.000000
mean,931.136556,0.491351,0.458169,0.141236,0.798013,0.211921,0.761589,0.582781,0.187334,0.026490,...,0.003311,0.003311,0.026490,0.009934,0.003311,0.003311,0.003311,0.013245,0.023179,0.039735
std,641.565472,0.173377,0.132400,0.098291,0.402149,0.409347,0.426819,0.493918,0.163478,0.160854,...,0.057544,0.057544,0.160854,0.099337,0.057544,0.057544,0.057544,0.114512,0.150721,0.195660
min,129.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,449.250000,0.402985,0.362637,0.062042,1.000000,0.000000,1.000000,0.000000,0.125000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,792.380000,0.507463,0.472527,0.124573,1.000000,0.000000,1.000000,1.000000,0.125000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,1225.635000,0.507463,0.538462,0.124573,1.000000,0.000000,1.000000,1.000000,0.250000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,3999.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [228]:
data_X = data_encoded.drop('price',1)
data_y = data_encoded['price']
x_train, x_test, y_train, y_test = train_test_split(data_X, data_y, test_size=0.33, random_state=28) 

### AdaBoost

Adaptive Boosting (AdaBoost) is a machine learning algorithm (ensamble learning method) that combines multiple weak learners to build one strong learner by progressively learning from other's wrongly classified objects. The weak learners in AdaBoost are decision trees with a single split (1 node and 2 leaves) called decision stumps. AdaBoost works by putting more weight assigned to the incorrectly classified samples so that they are classified correctly in the next decision stumps. AdaBoost works for both regression and classification problems.


In [229]:
#Ada Boost
def ada_boost_model(x_train, y_train, x_test, y_test):
    
    param_grid = {'n_estimators': [10, 50, 100, 500, 1000],
                  'learning_rate': [0.001, 0.003, 0.01, 0.03, 0.1, 0.3, 1, 3, 10]}
    
    gs_ab = GridSearchCV(AdaBoostRegressor(), param_grid = param_grid, scoring='neg_mean_absolute_error', cv=10)
    gs_ab.fit(x_train, y_train)
    
    best_ab_model = gs_ab.best_estimator_
    
    y_pred = best_ab_model.predict(x_test)
    
    mae_ab = mean_absolute_error(y_test, y_pred)
    
    print('Best training score: ', -gs_ab.best_score_)
    print(best_ab_model)
    print("\nMean Absolute Error for AdaBoost is: %.0f" %mae_ab)

In [230]:
ada_boost_model(x_train, y_train, x_test, y_test)

Best training score:  288.57183505975615
AdaBoostRegressor(base_estimator=None, learning_rate=0.03, loss='linear',
         n_estimators=100, random_state=None)

Mean Absolute Error for AdaBoost is: 238


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


In [156]:
# #XGBoost
# def xgboost(data):
#     data_X = data.drop('price',1)
#     data_y = data['price']
#     X_train, X_test, y_train, y_test = train_test_split(data_X, data_y, test_size=0.33, random_state=25) 
    
#     regressor = GradientBoostingRegressor()
#     model = regressor.fit(X_train, y_train)
#     y_pred = model.predict(X_test)

#     print("Mean Squared Error:", metrics.mean_absolute_error(y_test, y_pred))
#     print("R-Squared :", metrics.r2_score(y_test, y_pred))

### XGBoost
Extreme Gradient Boosting (XGBoost) is a scalable and improved version of the gradient boosting algorithm. It has better model performance, computational speed, and efficiency.

XGBoost has features which makes it more robust and flexible, such as:

 - Tree Pruning: Pruning prevents overfitting of training data. Pruing reduces the size of regression trees by replacing nodes that do not contribute to improving classification on leaves. XGBoost creates splits up to max_depth specified and starts pruning from backward until the loss is below a threshold.
 - Sparsity Awareness: XGBoost handles missing data by assigning them to default direction and finding the best imputation value that minimizes the training loss.


 



In [234]:
def xgboost_model(x_train, y_train, x_test, y_test):
#     dtrain = xgb.DMatrix(X_train, label=y_train)
#     dtest = xgb.DMatrix(X_test, label=y_test)
    
    param_grid = {'eta': [0.001,0.01, 0.1, 0.3, 0.5,0.7, 1], #step size shrinkage used to prevent overfitting. Range is [0,1]
                    'objective': ['reg:linear'], #loss function to be used like reg:linear for regression problems,
                    'n_estimators': [10, 50, 100, 500, 1000], #number of trees you want to build.
#                     'gamma':[i/10.0 for i in range(3,6)], #controls whether a given node will split based on the expected reduction in loss after the split. A higher value leads to fewer splits. Supported only for tree-based learners.
                      'subsample':[0.2, 0.5, 0.8],#percentage of samples used per tree. Low value can lead to underfitting.
                      'colsample_bytree':[0.2, 0.5, 0.8], #percentage of features used per tree. High value can lead to overfitting
                    'max_depth': [2,3,5]  #determines how deeply each tree is allowed to grow during any boosting round.
                    }
    
    regressor = XGBRegressor()
    gs_xgb = GridSearchCV(regressor, param_grid = param_grid, scoring='neg_mean_absolute_error', cv=10)
    gs_xgb.fit(x_train, y_train)
    
    best_xgb_model = gs_xgb.best_estimator_
    y_pred = best_xgb_model.predict(x_test)
    mae_xgb = mean_absolute_error(y_test, y_pred)
    
    print('Best training score: ', -gs_xgb.best_score_)
    print(best_xgb_model)
    print("\nMean Absolute Error for XGBoost is: %.0f" %mae_xgb)
    


In [235]:
xgboost_model(x_train, y_train, x_test, y_test)

# Best training score:  235.2391398484636
# XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
#        colsample_bynode=1, colsample_bytree=0.5, eta=0.01, gamma=0,
#        gpu_id=-1, importance_type='gain', interaction_constraints='',
#        learning_rate=0.00999999978, max_delta_step=0, max_depth=5,
#        min_child_weight=1, missing=nan, monotone_constraints='()',
#        n_estimators=1000, n_jobs=4, num_parallel_tree=1,
#        objective='reg:linear', random_state=0, reg_alpha=0, reg_lambda=1,
#        scale_pos_weight=1, subsample=0.8, tree_method='exact',
#        validate_parameters=1, verbosity=None)

# Mean Absolute Error for XGBoost is: 213

[23:42:55] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[23:42:55] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[23:42:55] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[23:42:55] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[23:42:55] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[23:42:55] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/objective/regression_obj.cu:17

[23:42:58] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[23:42:58] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[23:42:58] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[23:42:58] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[23:42:58] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[23:42:58] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/objective/regression_obj.cu:17

[23:43:04] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[23:43:04] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[23:43:05] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[23:43:05] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[23:43:05] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[23:43:06] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/objective/regression_obj.cu:17

[23:43:29] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[23:43:30] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[23:43:30] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[23:43:31] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[23:43:32] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[23:43:32] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/objective/regression_obj.cu:17

[23:43:35] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[23:43:36] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[23:43:36] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[23:43:36] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[23:43:36] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[23:43:36] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/objective/regression_obj.cu:17

...

[00:29:34] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[00:29:34] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[00:29:34] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[00:29:34] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[00:29:34] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[00:29:34] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/objective/regression_obj.cu:17

[00:29:37] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[00:29:37] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[00:29:37] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[00:29:37] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[00:29:37] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[00:29:37] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/objective/regression_obj.cu:17

[00:29:49] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[00:29:50] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[00:29:51] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[00:29:51] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[00:29:52] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[00:29:52] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/objective/regression_obj.cu:17

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


[00:30:33] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
Best training score:  235.2391398484636
XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=0.5, eta=0.01, gamma=0,
       gpu_id=-1, importance_type='gain', interaction_constraints='',
       learning_rate=0.00999999978, max_delta_step=0, max_depth=5,
       min_child_weight=1, missing=nan, monotone_constraints='()',
       n_estimators=1000, n_jobs=4, num_parallel_tree=1,
       objective='reg:linear', random_state=0, reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, subsample=0.8, tree_method='exact',
       validate_parameters=1, verbosity=None)

Mean Absolute Error for XGBoost is: 213


In [231]:
[i/10.0 for i in range(5,11)]

[0.5, 0.6, 0.7, 0.8, 0.9, 1.0]